In [2]:
# from __future__ import print_function
# import argparse
import torch
import torch.utils.data
import torch.nn as nn 
import torch.optim as optim
from torch.autograd import Variable  # change later
from torchvision import datasets, transforms
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt 
from torch.nn.parameter import Parameter # ?

from functools import reduce

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device


device(type='cpu')

##### Dataset

In [3]:
df = pd.read_csv("flchain.csv")
df

,Unnamed: 0,age,sex,sample.yr,kappa,lambda,flc.grp,creatinine,mgus,futime,death,chapter
0,1,97,F,1997,5.700,4.860,10,1.7,0,85,1,Circulatory
1,2,92,F,2000,0.870,0.683,1,0.9,0,1281,1,Neoplasms
2,3,94,F,1997,4.360,3.850,10,1.4,0,69,1,Circulatory
3,4,92,F,1996,2.420,2.220,9,1.0,0,115,1,Circulatory
4,5,93,F,1996,1.320,1.690,6,1.1,0,1039,1,Circulatory
...,...,...,...,...,...,...,...,...,...,...,...,...
7869,7870,52,F,1995,1.210,1.610,6,1.0,0,4997,0,NaN
7870,7871,52,F,1999,0.858,0.581,1,0.8,0,3652,0,NaN
7871,7872,54,F,2002,1.700,1.720,8,NaN,0,2507,0,NaN
7872,7873,53,F,1995,1.710,2.690,9,NaN,0,4982,0,NaN


In [4]:
# Shuffle data
df = df.sample(frac=1).reset_index(drop=True)
df

,Unnamed: 0,age,sex,sample.yr,kappa,lambda,flc.grp,creatinine,mgus,futime,death,chapter
0,4310,63,F,2001,1.87,2.10,9,1.0,0,3010,0,NaN
1,1830,72,F,1997,2.08,1.53,8,1.2,0,3449,1,Neoplasms
2,2228,70,M,1998,2.61,1.98,9,1.1,0,3934,0,NaN
3,5248,59,F,2003,1.31,1.95,7,NaN,0,2260,0,NaN
4,1711,73,F,1996,1.50,2.73,9,0.8,0,4649,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
7869,6813,54,M,1996,1.20,1.47,5,0.9,0,4126,1,Neoplasms
7870,7331,50,F,1997,1.20,2.11,7,1.0,0,4325,0,NaN
7871,1569,71,F,1996,1.91,1.21,7,0.8,0,4890,0,NaN
7872,6438,51,M,1995,1.09,1.16,3,1.2,0,4975,0,NaN


In [5]:
df.futime.describe()

count    7874.000000
mean     3661.042291
std      1432.677330
min         0.000000
25%      2852.000000
50%      4302.000000
75%      4773.000000
max      5215.000000
Name: futime, dtype: float64

In [6]:
print(df.shape)
print(df.dtypes)
print(df.isnull().sum())

(7874, 12)
Unnamed: 0      int64
age             int64
sex            object
sample.yr       int64
kappa         float64
lambda        float64
flc.grp         int64
creatinine    float64
mgus            int64
futime          int64
death           int64
chapter        object
dtype: object
Unnamed: 0       0
age              0
sex              0
sample.yr        0
kappa            0
lambda           0
flc.grp          0
creatinine    1350
mgus             0
futime           0
death            0
chapter       5705
dtype: int64


In [7]:
# Missing values processing
print(df['creatinine'].mean())
df['creatinine'] = df['creatinine'].fillna(df['creatinine'].mean())
df = df.drop(columns=['chapter'])
print(df.isnull().sum())

1.093516247700798
Unnamed: 0    0
age           0
sex           0
sample.yr     0
kappa         0
lambda        0
flc.grp       0
creatinine    0
mgus          0
futime        0
death         0
dtype: int64


In [8]:
# pd.get_dummies(df.sex)
df.sex.replace(['F', 'M'], [0, 1], inplace=True)
df.sex.value_counts()

0    4350
1    3524
Name: sex, dtype: int64

##### Train - test - validation split


In [9]:
get_x = lambda df: (df
                    .drop(columns=['Unnamed: 0', 'death', 'futime'])
                    .values.astype('float32'))

df_test = df.sample(frac=0.2)
df_train = df.drop(df_test.index)

X_train = get_x(df_train)
X_test = get_x(df_test)

Y_train = df_train[['death', 'futime']].to_numpy()
Y_test = df_test[['death', 'futime']].to_numpy()

Y_train


array([[   0, 3010],
       [   1, 3449],
       [   0, 3934],
       ...,
       [   0, 4890],
       [   0, 4975],
       [   0, 2507]])

In [10]:
D_in, H, D_out = X_train.shape[1], 128, 32    # D_out 32 ?
batch_size = 32
num_time_units = 24 # 24 month?
time_bin = 30   # 30?
n_epochs = 1
learning_rate = 1e-3


In [11]:
class survdl(nn.Module):
    def __init__(self, D_in, H, D_out, num_time_units):
        super(survdl, self).__init__()
        self.sigmoid = nn.Sigmoid()
        self.fc_layer = nn.Sequential(nn.Linear(D_in, H), nn.ReLU(), nn.Dropout(0.5), nn.Linear(H, D_out))
        self.fc_layer2 = nn.Linear(1, num_time_units)
        self.beta = Parameter(torch.Tensor(D_out, 1))
        self.beta.data.uniform_(-0.001, 0.001)  # initialization?
        
    def score_1(self, x):
        return torch.exp(x.mm(self.beta))  # hazard function - s1
    
    def score_2(self, score1):
        return self.sigmoid(self.fc_layer2(score1))
    
    def forward(self, x):
        new_x = self.fc_layer(x)
        score1 = self.score_1(new_x)
        score2 = self.score_2(score1)
        return score1, score2
    

##### Function for C-index

In [53]:
def unique_set(lifetime):
    a = lifetime.data.cpu().numpy()   # lifetime.data.cpu().numpy()
    t = np.unique(a)
    sort_idx = np.argsort(a)
    a_sorted = a[sort_idx]
    unq_first = np.concatenate(([True], a_sorted[1:] != a_sorted[:-1]))
    unq_count = np.diff(np.nonzero(unq_first)[0])
    unq_idx = np.split(sort_idx, np.cumsum(unq_count))
    return t, unq_idx
    
def log_parlik(lifetime, censor, score1):  
    t, H = unique_set(lifetime)
    keep_index = np.nonzero(censor.data.cpu().numpy())[0]  #censor = 1  #.data.cpu()
    H = [list(set(h)&set(keep_index)) for h in H]
    n = [len(h) for h in H]
    
    score1 = score1.detach().data.cpu().numpy()   # .data.cpu()   #?
    total = 0
    for j in range(len(t)):
        total_1 = np.sum(np.log(score1)[H[j]])
        m = n[j]
        total_2 = 0
        for i in range(m):
            subtotal = np.sum(score1[sum(H[j:],[])]) - (i*1.0/m)*(np.sum(score1[H[j]]))
            subtotal = np.log(subtotal)
            total_2 = total_2 + subtotal
        total = total + total_1 - total_2
        total = np.array([total])
    return torch.Tensor(total).type(torch.FloatTensor).to(device).view(-1,1)
        

def acc_pairs(censor, lifetime):
    noncensor_index = np.nonzero(censor.data.cpu().numpy())[0]
    lifetime = lifetime.data.cpu().numpy()
    acc_pair = []
    # print(noncensor_index)
    for i in noncensor_index:
        all_j =  np.array(range(len(lifetime)))[lifetime > lifetime[i]]
        acc_pair.append([(i,j) for j in all_j])
    if acc_pair:       #*Check if list is empty
        acc_pair = reduce(lambda x,y: x + y, acc_pair)
    else:
        return
    return acc_pair



def rank_loss(lifetime, censor, score2, t, time_bin): 
    # score2 (n(samples)*24) at time unit t = 1,2,...,24
    acc_pair = acc_pairs(censor, lifetime)
    if not acc_pair:  #*Check if list is empty    
        return 0
    lifetime = lifetime.data.cpu().numpy()
    total = 0
    for i,j in acc_pair:
        yi = (lifetime[i] >= (t-1) * time_bin) * 1
        yj = (lifetime[j] >= (t-1) * time_bin) * 1
        a = torch.ones(1).type(torch.FloatTensor).to(device)
        L2dist = torch.dist(score2[j, t-1] - score2[i, t-1], a, 2)
        total = total + L2dist* yi * (1-yj)
    return total


def C_index(censor, lifetime, score1):
    score1 = score1.detach().data.cpu().numpy()  #.data.cpu()  #?
    acc_pair = acc_pairs(censor, lifetime)
    prob = sum([score1[i] >= score1[j] for (i, j) in acc_pair])[0]*1.0/len(acc_pair)
    return prob

In [13]:
model = survdl(D_in, H, D_out, num_time_units).to(device)

optimizer = optim.Adam(model.parameters(), lr = learning_rate)


##### Training and evaluating

In [39]:
def train(epoch):
    model.train()
    train_loss = 0    
    # idx = np.random.permutation(X_train.shape[0])     
    j = 0
    while j < X_train.shape[0]:
        if j < X_train.shape[0] - batch_size:
            data = Variable(torch.from_numpy(X_train[j:(j + batch_size)])).type(torch.FloatTensor).to(device)
            lifetime = Variable(torch.from_numpy(Y_train[j:(j + batch_size),1])).type(torch.FloatTensor).to(device)
            censor = Variable(torch.from_numpy(Y_train[j:(j + batch_size),0])).type(torch.FloatTensor).to(device)
        else:
            data = Variable(torch.from_numpy(X_train[j:])).type(torch.FloatTensor).to(device)
            lifetime = Variable(torch.from_numpy(Y_train[j:,1])).type(torch.FloatTensor).to(device)
            censor = Variable(torch.from_numpy(Y_train[j:,0])).type(torch.FloatTensor).to(device)
            
        optimizer.zero_grad()
        score1, score2 = model(data)
        loss1 = log_parlik(lifetime, censor, score1)
        loss2 = []
        for t in range(num_time_units):
            loss2.append(rank_loss(lifetime, censor, score2, t+1, time_bin))
        loss2 = sum(loss2)
        loss = 1.0 * loss1 + 0.5 * loss2
        loss.backward()      
        train_loss = loss.data[0]
        optimizer.step()
        j += batch_size
    return train_loss*1.0 / X_train.shape[0]

def test(epoch):
    model.eval()
    test_loss = 0
    j = 0
    while j < X_test.shape[0]:
        # print('Current j', j)
        if j < X_test.shape[0] - batch_size:
            data = Variable(torch.from_numpy(X_test[j:(j + batch_size)])).type(torch.FloatTensor).to(device)
            lifetime = Variable(torch.from_numpy(Y_test[j:(j + batch_size),1])).type(torch.FloatTensor).to(device)
            censor = Variable(torch.from_numpy(Y_test[j:(j + batch_size),0])).type(torch.FloatTensor).to(device)
        else:
            data = Variable(torch.from_numpy(X_test[j:])).type(torch.FloatTensor).to(device)
            lifetime = Variable(torch.from_numpy(Y_test[j:,1])).type(torch.FloatTensor).to(device)
            censor = Variable(torch.from_numpy(Y_test[j:,0])).type(torch.FloatTensor).to(device)
        # y_pred = model(data)
        score1, score2 = model(data)
        loss1 = log_parlik(lifetime, censor, score1)
        loss2 = []
        for t in range(num_time_units):
            if j == 1568:
                print("T", t)
                print("Censor", censor)
                print("Lifetime", lifetime)
                print("Score2", score2)
                print("\n\n\n")
            loss2.append(rank_loss(lifetime, censor, score2, t+1, time_bin))
        loss2 = sum(loss2)
        loss = 1.0 * loss1 + 0.5 * loss2
        test_loss += loss.data[0]
        j += batch_size
    return test_loss*1.0 / X_test.shape[0]
    
for epoch in range(1, n_epochs + 1):
    train_loss = train(epoch)
    test_loss = test(epoch)
    print('====> Epoch: %d training loss: %.4f'%(epoch, train_loss))
    print('====> Epoch: %d testing loss: %.4f'%(epoch, test_loss))

# test_loss = test(epoch)
    

T 0
Censor tensor([0., 0., 0., 0., 0., 0., 0.])
Lifetime tensor([2513., 4656., 4143., 4927., 2239., 4618., 4665.])
Score2 tensor([[0.5592, 0.2640, 0.6621, 0.7327, 0.4260, 0.4132, 0.6319, 0.8083, 0.2374,
         0.3090, 0.8734, 0.3858, 0.9028, 0.8780, 0.4878, 0.2371, 0.3161, 0.4403,
         0.5058, 0.6787, 0.1938, 0.3503, 0.5484, 0.3977],
        [0.5590, 0.2646, 0.6618, 0.7325, 0.4261, 0.4135, 0.6315, 0.8081, 0.2375,
         0.3090, 0.8731, 0.3859, 0.9026, 0.8777, 0.4883, 0.2373, 0.3163, 0.4403,
         0.5056, 0.6786, 0.1940, 0.3503, 0.5485, 0.3976],
        [0.5590, 0.2646, 0.6618, 0.7325, 0.4261, 0.4135, 0.6315, 0.8081, 0.2375,
         0.3090, 0.8731, 0.3859, 0.9026, 0.8777, 0.4883, 0.2373, 0.3163, 0.4403,
         0.5056, 0.6786, 0.1940, 0.3503, 0.5485, 0.3976],
        [0.5589, 0.2649, 0.6617, 0.7324, 0.4261, 0.4136, 0.6313, 0.8080, 0.2375,
         0.3090, 0.8729, 0.3860, 0.9024, 0.8775, 0.4886, 0.2374, 0.3164, 0.4403,
         0.5055, 0.6786, 0.1941, 0.3503, 0.5486, 0.3975]

In [21]:
# train_loss = train(1)

In [40]:
# concordance - training
data_train = Variable(torch.from_numpy(X_train)).type(torch.FloatTensor).to(device)
lifetime_train = Variable(torch.from_numpy(Y_train[:,1])).type(torch.FloatTensor).to(device)  #0
censor_train = Variable(torch.from_numpy(Y_train[:,0])).type(torch.FloatTensor).to(device)  #1

score1_train, score2_train = model(data_train)
C_index_train = C_index(censor_train, lifetime_train, score1_train)
print('Concordance index for training data: {:.4f}'.format(C_index_train))


# concordance - test
data_test = Variable(torch.from_numpy(X_test)).type(torch.FloatTensor).to(device)
lifetime_test = Variable(torch.from_numpy(Y_test[:,1])).type(torch.FloatTensor).to(device) #0
censor_test = Variable(torch.from_numpy(Y_test[:,0])).type(torch.FloatTensor).to(device) #1

score1_test, score2_test = model(data_test)
C_index_test = C_index(censor_test, lifetime_test, score1_test)
print('Concordance index for test data: {:.4f}'.format(C_index_test))

Concordance index for training data: 0.2231
Concordance index for test data: 0.2034


In [15]:
Y_test

array([[   0, 3564],
       [   0, 4621],
       [   1, 3533],
       ...,
       [   0, 2239],
       [   0, 4618],
       [   0, 4665]])

In [41]:
print(data_train[:15])
print

tensor([[6.3000e+01, 0.0000e+00, 2.0010e+03, 1.8700e+00, 2.1000e+00, 9.0000e+00,
         1.0000e+00, 0.0000e+00],
        [7.2000e+01, 0.0000e+00, 1.9970e+03, 2.0800e+00, 1.5300e+00, 8.0000e+00,
         1.2000e+00, 0.0000e+00],
        [7.0000e+01, 1.0000e+00, 1.9980e+03, 2.6100e+00, 1.9800e+00, 9.0000e+00,
         1.1000e+00, 0.0000e+00],
        [5.9000e+01, 0.0000e+00, 2.0030e+03, 1.3100e+00, 1.9500e+00, 7.0000e+00,
         1.0935e+00, 0.0000e+00],
        [7.3000e+01, 0.0000e+00, 1.9960e+03, 1.5000e+00, 2.7300e+00, 9.0000e+00,
         8.0000e-01, 0.0000e+00],
        [6.8000e+01, 0.0000e+00, 1.9970e+03, 8.1700e-01, 4.4600e-01, 1.0000e+00,
         1.0000e+00, 1.0000e+00],
        [5.1000e+01, 0.0000e+00, 1.9960e+03, 1.8800e+00, 3.3400e+00, 1.0000e+01,
         1.0000e+00, 0.0000e+00],
        [6.0000e+01, 0.0000e+00, 1.9960e+03, 1.0200e+00, 1.4400e+00, 4.0000e+00,
         8.0000e-01, 0.0000e+00],
        [8.0000e+01, 0.0000e+00, 1.9960e+03, 3.2200e+00, 4.0900e+00, 1.0000e+01,

In [43]:
print(censor_train[:6])
print(lifetime_train[:6])
print(score1_train[:6])

tensor([0., 1., 0., 0., 0., 0.])
tensor([3010., 3449., 3934., 2260., 4649., 4305.])
tensor([[1.6189],
        [1.6145],
        [1.6157],
        [1.6217],
        [1.6138],
        [1.6182]], grad_fn=<SliceBackward0>)


In [46]:
C_index(censor_train[:500], lifetime_train[:500], score1_train[:500])

0.20856186338058266

In [75]:
def unique_set(lifetime):
    a = lifetime.numpy()   # lifetime.data.cpu().numpy()
    t, idx = np.unique(a, return_index=True)
    sort_idx = np.argsort(a)
    a_sorted = a[sort_idx]
    unq_first = np.concatenate(([True], a_sorted[1:] != a_sorted[:-1]))
    unq_count = np.diff(np.nonzero(unq_first)[0])
    unq_idx = np.split(sort_idx, np.cumsum(unq_count))
    return idx, unq_idx
unique_set(torch.Tensor([1, 1, 2, 5,7, 4]))

(array([0, 2, 5, 3, 4]),
 [array([0, 1]), array([2]), array([5]), array([3]), array([4])])

In [63]:
a_sorted=np.array([1,1,3])
np.concatenate(([True], a_sorted[1:] != a_sorted[:-1]))

array([ True, False,  True])